In [30]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

# Data Preprocessing Functions
def preprocess_data(df):
    # Replace '?' with NaN for easy handling of missing values
    df = df.replace('?', np.nan)

    # Impute missing values with the most common value for each column
    imputer = SimpleImputer(strategy='most_frequent')
    df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

    # Convert categorical columns to numerical using label encoding
    categorical_columns = df.select_dtypes(include=['object']).columns
    for column in categorical_columns:
        df[column] = pd.Categorical(df[column]).codes

    return df

def main():
    print("Loading the data ie. test and train into the dataframes")
    # Load the data
    train_data = pd.read_csv('train_final.csv')
    test_data = pd.read_csv('test_final.csv')

    print("Preprocessing the data...")
    # Preprocess the data
    train_data = preprocess_data(train_data)
    test_data = preprocess_data(test_data)

    # Ensure the target column 'income>50K' is present in the training data
    if 'income>50K' not in train_data.columns:
        raise KeyError("'income>50K' column not found in training data")

    # Split features and target
    X_train = train_data.drop(columns='income>50K').values
    y_train = train_data['income>50K'].values

    # Handle the test data
    if 'income>50K' in test_data.columns:
        X_test = test_data.drop(columns='income>50K').values
    else:
        X_test = test_data.drop(columns='ID', errors='ignore').values

    test_ids = test_data.get('ID', pd.Series(dtype='int'))

    print("Training the decision tree...")
    # Train the Decision Tree
    dt_model = DecisionTreeClassifier(max_depth=10, random_state=0)
    dt_model.fit(X_train, y_train)

    print("Predicting probabilities for the test set...")
    # Predict probabilities for test data using Decision Tree
    predictions = dt_model.predict_proba(X_test)[:, 1]  # Probability of class "income>50K=1"

    print("Generating file that is to be submitted ...")
    # Prepare submission with IDs starting from 1
    submission = pd.DataFrame({'ID': test_ids + 1, 'Prediction': predictions})
    submission.to_csv('Submission_Decision_Tree.csv', index=False)
    print("File Created Successfully!!!")

if __name__ == "__main__":
    main()

Loading the data...
Preprocessing the data...
Training the decision tree...
Predicting probabilities for the test set...
Preparing submission file...
Submission file 'submission_decision_tree.csv' created successfully.
